In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('./data.csv')

In [20]:
df.describe()

,lat,lng,no_bath,no_bed,no_toilets,price
count,7455.000000,7455.000000,7455.000000,7455.000000,7455.000000,7.455000e+03
mean,6.530563,1291.431622,2.786050,2.786050,2.786050,1.898320e+06
std,0.517048,22666.378468,1.153834,1.153834,1.153834,4.434080e+06
min,6.409760,-122.271114,1.000000,1.000000,1.000000,1.000000e+00
25%,6.453528,3.348967,2.000000,2.000000,2.000000,5.000000e+05
50%,6.507098,3.384168,3.000000,3.000000,3.000000,8.000000e+05
75%,6.583560,3.469745,4.000000,4.000000,4.000000,1.600000e+06
max,37.804364,400106.000000,10.000000,10.000000,10.000000,1.760000e+08


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7455 entries, 0 to 7454
Data columns (total 7 columns):
Unnamed: 0    7455 non-null int64
lat           7455 non-null float64
lng           7455 non-null float64
no_bath       7455 non-null int64
no_bed        7455 non-null int64
no_toilets    7455 non-null int64
price         7455 non-null int64
dtypes: float64(2), int64(5)
memory usage: 407.8 KB


In [4]:
df = df.drop("Unnamed: 0", axis=1)

In [5]:
data = df.values

In [6]:
data.shape

(7455, 6)

In [7]:
X = data[:, 0:5]
Y = data[:, 5]

In [21]:
def baseline_model():
    model = Sequential()
    model.add(Dense(250, input_dim=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [22]:
seed = 7
numpy.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [16]:
lekki = pd.DataFrame({ 'lng': [6.4698], 'lat': [3.5852], 'no_bed': [1], 'no_toilets': [1], 'no_bath': [1] })

In [17]:
lekki = lekki.values

In [19]:
lekki

array([[3.5852, 6.4698, 1.    , 1.    , 1.    ]])

In [13]:
estimator.fit(X, Y)

In [18]:
estimator.predict(lekki)

array(903791.44, dtype=float32)

In [14]:
estimator.model.save("model.h5")

In [ ]:
!tensorflowjs_converter --input_format=keras model.h5 tf_js_model